# Traitement motifs frequents

In [6]:
import openpyxl
import csv
import pandas as pd

###  Importer les données 

In [7]:
df_all= pd.DataFrame()
for file in os.listdir("C:\\Users\\actou\\OneDrive\\Documents\\IODAA\\AZOTE\\test extraction de motifs\\rulesnewfiles\\"):
    if file.endswith(".csv"):
        df = pd.read_csv("C:\\Users\\actou\\OneDrive\\Documents\\IODAA\\AZOTE\\test extraction de motifs\\rulesnewfiles\\" + file)
    elif file.endswith(".xlsx"):
        df = pd.read_excel("C:\\Users\\actou\\OneDrive\\Documents\\IODAA\\AZOTE\\test extraction de motifs\\rulesnewfiles\\" + file)
        df_all = df_all.append(df)
        df_all['file'] = file
df = df_all 

C:\Users\actou\AppData\Local\Temp\ipykernel_33960\3601447534.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_all.append(df)
C:\Users\actou\AppData\Local\Temp\ipykernel_33960\3601447534.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_all.append(df)
C:\Users\actou\AppData\Local\Temp\ipykernel_33960\3601447534.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_all.append(df)


### Cleaning 

In [8]:
df.dropna(axis='index', inplace=True)
df.drop(0,inplace=True)
df.head()
UNIQUE = pd.DataFrame(df["Consequent"].unique(), columns = ["UNIQUE"])
UNIQUE


,UNIQUE
0,Pic_J182=1
1,Pic_J45=1
2,"CP=ble, Pic_J182=1"
3,"Peuplement=ble, Pic_J182=1"
4,"Travail_sol_J90=False, Pic_J182=1"
5,"Travail_sol_J90=False, Pic_J45=1"
6,"Ajout_phyto_30J=0, Pic_J45=1"
7,"CP=ble, Pic_J45=1"
8,"Peuplement=ble, Pic_J45=1"
9,"CP=colza, Pic_J45=1"


### Méthodologie :
- **Etape 1**  Garder les lignes avec une valeur de support supérieure à un seuil 
- **Etape 2**  Extraire la liste de varaibles qui apparaissent dans les motifs et compter leur frequence de retour 


In [23]:
import xlwt 
  
# create a workbook 
wb = xlwt.Workbook() 
sheet1 = wb.add_sheet('Sheet 1') 
wb.save('liste_motifs.xlsx') 
  
# create a workbook 
wb_2 = xlwt.Workbook() 
sheet1 = wb_2.add_sheet('Sheet 1') 
wb.save('results_liste_motifs.xlsx') 


In [48]:
def analyze_motifs(df,seuil,target):
    '''
        Cette fonction prend en entrée :
            - un output de ORANGE DATA MINING - Association Rules 
            - Une vakeur support seuil 

        Elle renvoie 
            - d'extrire la liste des motifs au dessus d'un seuil donné
            - d'extraire la liste es varaibles te modalités qui revienent dans ces motifs ainsi que leurs frequences 
    '''

    # Liste des couples "Varaibles , modalités --> dataframe df_Antecedent
    df_cible = df[df['Consequent'] == target]
    df_cible = df_cible[df_cible["Support"] >= seuil]

    Variables, Modalités =[],[]
    for i in df_cible['Antecedent']:
        var_mod = i.split(', ')
        for j in var_mod:
            var, mod = j.split('=')
            Variables.append(var)
            Modalités.append(mod)

    df_Antecedent = pd.DataFrame({'Variable': Variables, 'Modalité': Modalités})


    # Création d'un dictionnaire avec les couples uniques et leurs fréquences
    dic = {}
    for i, j in zip(df_Antecedent['Variable'], df_Antecedent['Modalité']):
        if (i, j) in dic:
            dic[(i, j)] += 1
        else:
            dic[(i, j)] = 1

    # Création du dataframe de résultats
    df_result = pd.DataFrame({'Variables': list(dic.keys()), 'Modalités': list(dic.values())})
    df_result['Frequences'] = df_result['Modalités']
    df_result.drop('Modalités', axis = 1, inplace = True)
    df_result['NB_Motifs'] = df.shape[0]
    
    # Afficher les resultats 
    print(df_result.sort_values('Frequences', ascending=False))
    print(df[["Support","Antecedent","Consequent"]])
    
    #df_cible.to_excel("liste_motifs.xlsx", sheet_name=target[:30],header=True,index=False, startrow=0, startcol=0)
    #df_result.to_excel('results_iste_motifs.xlsx', sheet_name=target[:30],header=True,index=False, startrow=0, startcol=0)

In [49]:
analyze_motifs(df,0.1,'Pic_J182=1')

                  Variables  Frequences  NB_Motifs
2              (Pic_J45, 1)          17       1194
0      (Ajout_ferti_30J, 1)           6       1194
3  (Travail_sol_J90, False)           6       1194
1     (Temp_quali, moyenne)           5       1194
4      (Ajout_phyto_30J, 0)           5       1194
7                 (CP, ble)           3       1194
6         (Peuplement, ble)           2       1194
5       (Saison, printemps)           1       1194
    Support                                         Antecedent  \
2     0.064  CP=ble, Ajout_ferti_30J=1, Temp_quali=moyenne,...   
3     0.064  Peuplement=ble, Ajout_ferti_30J=1, Temp_quali=...   
4     0.064  CP=ble, Peuplement=ble, Ajout_ferti_30J=1, Tem...   
5     0.064  CP=ble, Travail_sol_J90=False, Ajout_ferti_30J...   
6     0.064  Peuplement=ble, Travail_sol_J90=False, Ajout_f...   
..      ...                                                ...   
752   0.126  Travail_sol_J90=False, Temp_quali=moyenne, Pic...   
753   0.121  

In [50]:
analyze_motifs(df,0.1,'Pic_J45=1')

                     Variables  Frequences  NB_Motifs
2                (Pic_J182, 1)          20       1194
3     (Travail_sol_J90, False)           8       1194
4         (Ajout_ferti_30J, 1)           6       1194
0         (Ajout_phyto_30J, 0)           5       1194
1        (Temp_quali, moyenne)           5       1194
7                    (CP, ble)           3       1194
5  (Pluie_quali, sup a la moy)           2       1194
8            (Peuplement, ble)           2       1194
6          (Saison, printemps)           1       1194
    Support                                         Antecedent  \
2     0.064  CP=ble, Ajout_ferti_30J=1, Temp_quali=moyenne,...   
3     0.064  Peuplement=ble, Ajout_ferti_30J=1, Temp_quali=...   
4     0.064  CP=ble, Peuplement=ble, Ajout_ferti_30J=1, Tem...   
5     0.064  CP=ble, Travail_sol_J90=False, Ajout_ferti_30J...   
6     0.064  Peuplement=ble, Travail_sol_J90=False, Ajout_f...   
..      ...                                                ...  

In [ ]:
''' 
df_182 = df[df['Consequent'] == 'Pic_J182=1']
df_182["Support"].describe()
df_182 = df_182[df_182["Support"] >= 0.1]

Variables, Modalités =[],[]
for i in df_182 ['Antecedent']:
    var_mod = i.split(', ')
    for j in var_mod:
        var, mod = j.split('=')
        Variables.append(var)
        Modalités.append(mod)

df_Antecedent = pd.DataFrame({'Variable': Variables, 'Modalité': Modalités})
# Création d'un dictionnaire avec les couples uniques et leurs fréquences
dic = {}
for i, j in zip(df_Antecedent['Variable'], df_Antecedent['Modalité']):
    if (i, j) in dic:
        dic[(i, j)] += 1
    else:
        dic[(i, j)] = 1

# Création du dataframe de résultats
df_result = pd.DataFrame({'Variables': list(dic.keys()), 'Modalités': list(dic.values())})
df_result['Frequences'] = df_result['Modalités']
df_result.drop('Modalités', axis = 1, inplace = True)
df_result['NB_Motifs'] = df_182.shape[0]
df_result.sort_values('Frequences', ascending=False)
'''